# **Задание**

Задание состоит из двух частей.

# Часть 1. Генерация информации
Используя библиотеку для генерации логов веб-сервера сгенерируйте 100 000 записей логов и сохраните их в CSV-файл.

Логи содержат следующую информацию:

IP-адрес клиента

Временная метка запроса

HTTP-метод (GET, POST, etc.)

URL запроса

Код ответа (200, 404, etc.)

Размер ответа в байтах

Сгенерировали 100,000 записей логов и сохранили их в CSV-файл.


# Часть 2. Анализ информации
1. Сгруппируйте данные по IP и посчитайте количество запросов для каждого IP, выводим 10 самых активных IP. Формат вывода, как на скрине ниже.

2. Сгруппируйте данные по HTTP-методу и посчитайте количество запросов для каждого метода.

3. Профильтруйте и посчитайте количество запросов с кодом ответа 404.

4. Сгруппируйте данные по дате и просуммируйте размер ответов, сортируйте по дате.

In [1]:
# Сгенерируем 100 000 записей логов и сохраним их в CSV-файл:

!pip install faker
import csv
from faker import Faker
import random

fake = Faker()

num_records = 100000

http_methods = ['GET', 'POST', 'PUT', 'DELETE']
response_codes = [200, 301, 404, 500]

file_path = "web_server_logs.csv"

with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ip', 'timestamp', 'method', 'url', 'response_code', 'response_size'])

    for _ in range(num_records):
        ip = fake.ipv4()
        timestamp = fake.date_time_this_year().isoformat()
        method = random.choice(http_methods)
        url = fake.uri_path()
        response_code = random.choice(response_codes)
        response_size = random.randint(100, 10000)

        writer.writerow([ip, timestamp, method, url, response_code, response_size])

print(f"Сгенерировано {num_records} записей и сохранено в {file_path}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.3 MB/s eta 0:00:00
Сгенерировано 100000 записей и сохранено в web_server_logs.csv


In [2]:
pip --version

pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


In [3]:
#Устанавливаем pySpark
!pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=98ea62e8e3fe1491cd77950388d33015109ad5fcea4fe6520b8d6aa4f225ca1e
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [4]:
# Импорт необходимых библиотек
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum, to_date

In [5]:
# Создание SparkSession
spark = SparkSession.builder.appName('Logs').getOrCreate()

In [6]:
# Установка уровня логирования на "ERROR"
spark.sparkContext.setLogLevel("ERROR")

In [7]:
# Чтение данных
logs_df = spark.read.csv('/content/web_server_logs.csv', header=True, inferSchema=True)

In [8]:
# 1. Сгруппируем данные по IP и посчитаем количество запросов для каждого IP, выводим 10 самых активных IP:

top_addresses = logs_df.groupBy('ip').agg(count(col('timestamp')).alias('request_count')).orderBy(col('request_count').desc()).limit(10)
print('Top 10 active IP addresses:')
top_addresses.show()

Top 10 active IP addresses:
+---------------+-------------+
|             ip|request_count|
+---------------+-------------+
|  76.67.201.232|            2|
|   5.170.144.66|            1|
|    83.122.4.83|            1|
| 140.191.48.149|            1|
| 171.168.215.28|            1|
|   95.138.3.198|            1|
| 19.215.209.222|            1|
|145.205.208.170|            1|
|  82.101.42.156|            1|
|203.199.245.254|            1|
+---------------+-------------+



In [9]:
# 2.Сгруппируем данные по HTTP-методу и посчитаем количество запросов для каждого метода:

requests_count = logs_df.groupBy('method').agg(count(col('method')).alias('method_count'))
print('Requests count by HTTP method:')
requests_count.show()

Requests count by HTTP method:
+------+------------+
|method|method_count|
+------+------------+
|  POST|       25036|
|DELETE|       24954|
|   PUT|       25063|
|   GET|       24947|
+------+------------+



In [10]:
# 3.Отфильтруем данные и посчитаем количество запросов с кодом ответа 404:

responses_404 = logs_df.filter(col('response_code') == 404).count()
print(f'Number of 404 response codes: {responses_404}')

Number of 404 response codes: 25300


In [11]:
# 4.Сгруппируем данные по дате и просуммируем размер ответов, сортировка по дате.

# Преобразуем столбец timestamp  в формат даты:
logs_df = logs_df.withColumn('timestamp', to_date(col('timestamp'), "yyyy-MM-dd"))

# Проверяем типы данных:
logs_df.dtypes

[('ip', 'string'),
 ('timestamp', 'date'),
 ('method', 'string'),
 ('url', 'string'),
 ('response_code', 'int'),
 ('response_size', 'int')]

In [12]:
total_response_size = logs_df.groupBy(col('timestamp').alias('date')).agg(sum(col('response_size')).alias('total_response_size')).orderBy('date')
print('Total response size by day:')
total_response_size.show()

Total response size by day:
+----------+-------------------+
|      date|total_response_size|
+----------+-------------------+
|2024-01-01|            2006720|
|2024-01-02|            1922129|
|2024-01-03|            1920184|
|2024-01-04|            1883820|
|2024-01-05|            1923729|
|2024-01-06|            1863618|
|2024-01-07|            1840074|
|2024-01-08|            1969014|
|2024-01-09|            1680912|
|2024-01-10|            1841812|
|2024-01-11|            1843664|
|2024-01-12|            1673963|
|2024-01-13|            1753178|
|2024-01-14|            1849427|
|2024-01-15|            1792300|
|2024-01-16|            1741818|
|2024-01-17|            1739419|
|2024-01-18|            1806903|
|2024-01-19|            1763054|
|2024-01-20|            1755199|
+----------+-------------------+
only showing top 20 rows

